In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra""
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [34]:
from keras.models import Sequential, load_model
from keras.layers import Dense
import keras as keras
import math

## Simple Neural Network Model made using Keras

In [35]:
model = Sequential()
model.add(keras.Input(shape = (None,50)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation="sigmoid"))
model.compile(loss='mse', optimizer='adam', steps_per_execution = 32)

In [36]:
epsilon = 0.05
gamma = 0.1

In [37]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [38]:
def act(stock_data,t, size):
    if(t>size):
        state = stock_data[t-size:t+1]
    else:
        state = [stock_data[0] for i in range(size-t)] + list(stock_data[:t+1])
    stat = []
    for i in range(len(state)-1):
        stat.append(sigmoid(state[i+1] - state[i]))
    state = np.array(stat)
    if(np.random.rand() <= epsilon):
        return np.random.rand(3)
    else:
        return model.predict(np.array([state]))[0]

In [39]:
def feedback(stock_data, temp_vec, t, size):
    if(t>size):
        state = stock_data[t-size:t+1]
    else:
        state = [stock_data[0] for i in range(size-t)] + list(stock_data[:t+1])
    stat = []
    for i in range(len(state)-1):
        stat.append(sigmoid(state[i+1] - state[i]))
    state = np.array(stat)
    model.fit(np.array([state]), np.array([temp_vec]), epochs=1, verbose=0)
    return

In [40]:
def train(episode_count, stock_data):
    for e in range(episode_count+1):
        profit = 0
        total_hold = 0
        new_vec = act(stock_data, 0, 50)
        print('EPISODE '+str(e))
        for t in range(len(stock_data)-2):
            action_vec = new_vec
            action = np.argmax(action_vec)
            if action==0:  ##BUY
                print('Time '+str(t)+' : BUY')
                profit -= stock_data[t]
                total_hold += 1
                reward = stock_data[t+1] - stock_data[t]
            elif action==1:  ##SELL
                print('Time '+str(t)+' : SELL')
                profit += stock_data[t]
                total_hold -= 1
                reward = stock_data[t] - stock_data[t+1]
            else:  ##HOLD
                print('Time '+str(t)+' : HOLD')
                reward = 0
            new_vec = act(stock_data, t+1, 50)
            target = reward + gamma*np.max(new_vec)
            temp_vec = action_vec
            temp_vec[np.argmax(temp_vec)] = target
            feedback(stock_data, temp_vec, t, 50)
        profit += total_hold*stock_data[-1]
        print('EPISODE '+str(e)+' PROFIT IS '+str(profit))

In [41]:
!pip install yfinance

In [42]:
import yfinance as yf

In [43]:
stock_data = yf.download(tickers = 'GOOGL', period = '100000d', interval = '1d')
stock_data = stock_data['Close']
if(len(stock_data) >1000):
    stock_data = stock_data[-1000:]
stock_data

In [44]:
model.summary()

In [45]:
train(5, stock_data)

In [51]:
model.save('model.h5')

In [52]:
os.chdir(r'/kaggle/working')

In [53]:
!zip -r file.zip './'

In [54]:
from IPython.display import FileLink
FileLink('./file.zip')